# Introdução

Esse Notebook será responsável pelo preprocessamento dos dados contidos em ./data/raw para formatos compatíveis e otimizados para o treinamento de cada modelo
As definições dos problemas aos quais os modelos deverão solucionar já foram definidas no notebook "Coleta de Dados"

Modelos a serem Criados:

1. Modelo Linear: MLP sem funções de ativação, composta apenas de somas lineares
2. MLP: rede neural - efetivamente identica ao modelo linear, no entanto, apresenta função de ativação ao final do somatório de funções lineares
3. LSTM: Um modelo de rede neural recorrente, com capacidade de diferenciação de informação de curto e longo prazo
4. TFT: Modelo baseado em LLMs desenvolvido pela microsoft - servirá como um comparativo mais moderno


## Dependências

In [17]:
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "1"
# Dependências mínimas para TFT — simples e com foco em GPU
try:
    import torch
except Exception:
    import shutil
    has_gpu = shutil.which("nvidia-smi") is not None
    if has_gpu:
        # tenta instalar com suporte CUDA (ajuste a versão cu de acordo com sua stack, p.ex. cu121)
        %pip install -q torch --index-url https://download.pytorch.org/whl/cu121
    else:
        %pip install -q torch --index-url https://download.pytorch.org/whl/cpu
    # libs do pipeline TFT
    %pip install -q pytorch-lightning pytorch-forecasting

    import torch



print(f"torch={torch.__version__} | cuda={torch.cuda.is_available()}")

print("Verificando dependências (pyarrow para Parquet)...")

try:
    import pyarrow as pa
    print(f"PyArrow disponível: {pa.__version__}")
except Exception:
    print("Instalando pyarrow...")
    !pip install --upgrade "pyarrow>=18" --quiet
    import importlib
    importlib.invalidate_caches()
    import pyarrow as pa
    print(f"PyArrow instalado: {pa.__version__}")

# fastparquet é opcional
try:
    import fastparquet  # noqa: F401
    print("fastparquet disponível (opcional)")
except Exception:
    pass

# Outras bibliotecas sob demanda
for lib in [
    "numpy", "python-dotenv", "pandas", "matplotlib", "seaborn",
    "scikit-learn", "tensorflow[and-cuda]", "keras", "lxml", "pytz"
]:
    try:
        __import__(lib)
    except ImportError:
        print(f"Instalando {lib}...")
        !pip install {lib} --quiet

print("Dependências prontas")

torch=2.5.1+cu121 | cuda=True
Verificando dependências (pyarrow para Parquet)...
PyArrow disponível: 22.0.0
Instalando python-dotenv...
Instalando scikit-learn...
Instalando tensorflow[and-cuda]...
Dependências prontas


## VARIÁVEIS NECESSÁRIAS

In [18]:
# Imports para a API e utilidades
import os
from dotenv import load_dotenv
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras import mixed_precision
import tensorflow as tf
import pandas as pd

# Silenciando Warnings
import warnings
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=RuntimeWarning)
warnings.filterwarnings("ignore", category=Warning)  # last resort
warnings.filterwarnings("ignore", category=pd.errors.PerformanceWarning)



# ==============================================
# GPU CONFIGURATION
# ==============================================
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    for gpu in gpus:
        tf.config.experimental.set_memory_growth(gpu, True)
    mixed_precision.set_global_policy('mixed_float16')
    print(f"✅ GPU detected ({gpus[0].name}) - using mixed precision.")
else:
    print("⚠️ No GPU detected, running on CPU.")

# Carregar variáveis de ambiente do .env
load_dotenv()
# ---------------- CONFIG ---------------- #
COUNTRY_DOMAINS = {
    "FR": {"domain": "10YFR-RTE------C"},
    "ES": {"domain": "10YES-REE------0"},
    "PT": {"domain": "10YPT-REN------W"}
}

DATA_ITEMS = [
    {'key': 'load_total', 'documentType': 'A65', 'processType': 'A16', 'domainParam': 'outBiddingZone_Domain', 'parser': 'load'},
    {'key': 'market_prices', 'documentType': 'A44', 'processType': 'A07', 'domainParamIn': 'in_Domain', 'domainParamOut': 'out_Domain', 'parser': 'price'}
]

ENTSOE_TOKEN = os.environ.get("ENTSOE_SECURITY_TOKEN")
BASE_URL = "https://web-api.tp.entsoe.eu/api"
MAX_WORKERS = 100
RAW_DIR = os.path.join("data", "raw")
PARQUET_COMPRESSION = "zstd"
os.makedirs(RAW_DIR, exist_ok=True)

# ==============================================
# DICIONÁRIO DE PROBLEMAS
# ==============================================
problemas = [
    dict(name="treinamento", data_dir="data/treinamento", feats=["country","datetime","quantity_MW"], tgts=["quantity_MW"], lag=10*24, lead=3*24, vals=["quantity_MW"], countries=COUNTRY_DOMAINS.keys()),
    dict(name="N1A", data_dir="data/N1A", feats=["country","datetime","quantity_MW"], tgts=["quantity_MW"], lag=10*24, lead=3*24, vals=["quantity_MW"], countries=["ES"], reduced_dim = 3*24, mask_value=-999.0),
    dict(name="N1B", data_dir="data/N1B", feats=["country","datetime","quantity_MW"], tgts=["quantity_MW"], lag=10*24, lead=3*24, vals=["quantity_MW"], countries=["ES"], reduced_dim = 7*24, mask_value=-999.0),
    dict(name="N1C", data_dir="data/N1C", feats=["country","datetime","quantity_MW"], tgts=["quantity_MW"], lag=10*24, lead=3*24, vals=["quantity_MW"], countries=["ES"]),
    dict(name="N2A", data_dir="data/N2A", feats=["country","datetime","quantity_MW"], tgts=["quantity_MW"], lag=10*24, lead=3*24, vals=["quantity_MW"], countries=COUNTRY_DOMAINS.keys(), reduced_dim = 3*24, mask_value=-999.0),
    dict(name="N2B", data_dir="data/N2B", feats=["country","datetime","quantity_MW"], tgts=["quantity_MW"], lag=10*24, lead=3*24, vals=["quantity_MW"], countries=COUNTRY_DOMAINS.keys(), reduced_dim = 7*24, mask_value=-999.0),
    dict(name="N2C", data_dir="data/N2C", feats=["country","datetime","quantity_MW"], tgts=["quantity_MW"], lag=10*24, lead=3*24, vals=["quantity_MW"], countries=COUNTRY_DOMAINS.keys()),
]


✅ GPU detected (/physical_device:GPU:0) - using mixed precision.


# Capítulo 1: Préprocessamento de dados

Etapa de contrução da pipelines de pre-processamento de dados


## Classe geral de preprocessamento

In [19]:
import os
from typing import Optional, List, Tuple
import pandas as pd
from sklearn.preprocessing import MinMaxScaler, StandardScaler, LabelEncoder
import numpy as np
import time

class Preprocessor:
    """Pré-processador base.

    - lag/lead como inteiros são expandidos para ranges [1..N] quando apropriado.
    - feature_cols/target_cols definem bases permitidas e servem como seleção no export.
    - Nenhuma coluna é removida dos dados; seleção ocorre apenas na exportação.
    """
    def __init__(
        self,
        lag: int,
        lead: int,
        country_list: Optional[List[str]] = None,
        *,
        model_name: str = "linear",
        data_dir: str = "data/processed",
        feature_cols: Optional[List[str]] = None,
        target_cols: Optional[List[str]] = None,
    ):
        self.lag = lag
        self.lead = lead
        self.country_list = country_list
        self.model_name = model_name
        self.data_dir = data_dir
        self.save_dir = self.data_dir
        os.makedirs(self.save_dir, exist_ok=True)

        self.feature_cols: List[str] = list(feature_cols) if feature_cols else []
        self.target_cols: List[str] = list(target_cols) if target_cols else []

        self.norm_objects = {}
        self.encod_objects = {}
        self.df_base = pd.DataFrame()

    def _expand_steps(self, steps, default_max: Optional[int]) -> List[int]:
        """Normaliza passos: int→[1..N], None→[1..default_max], lista→como está."""
        if isinstance(steps, int):
            return list(range(1, steps + 1)) if steps > 0 else [1]
        if steps is None and isinstance(default_max, int) and default_max > 0:
            return list(range(1, default_max + 1))
        if isinstance(steps, (list, tuple)):
            return list(steps)
        return [1]

    def load_data(self, raw_dir: Optional[str] = None) -> pd.DataFrame:
        """Carrega Parquet unificado em data/raw (ou raw_dir) e atualiza self.df_base."""
        base_raw = raw_dir or os.path.join('data', 'raw')
        unified_path = os.path.join(base_raw, f'raw_dataset.parquet')
        if not os.path.exists(unified_path):
            raise FileNotFoundError(f"Arquivo unificado não encontrado: {unified_path}. Execute a coleta primeiro.")
        df = pd.read_parquet(unified_path, engine='pyarrow')
        if 'datetime' in df.columns:
            df['datetime'] = pd.to_datetime(df['datetime'], utc=True)
        if self.country_list and 'country' in df.columns:
            df = df[df['country'].isin(self.country_list)].copy()
        sort_cols = [c for c in ['country', 'datetime'] if c in df.columns]
        if sort_cols:
            df = df.sort_values(sort_cols).reset_index(drop=True)
            
        # Filtrando Colunas apenas para as necessárias
        cols = list(set([c for c in self.feature_cols + self.target_cols if c in df.columns]))
        df = df.loc[:, ~df.columns.duplicated()]  # optional: remove duplicates
        df = df[cols]

        self.df_base = df
        return self.df_base

    def encode(self, encode_cols: str = 'datetime', encode_method: str = 'label') -> pd.DataFrame:
        """Codifica de forma não destrutiva e atualiza self.df_base.

        - label: usa LabelEncoder com suporte a NaN via placeholder interno que é revertido no decode.
        - time_cycle: adiciona features de calendário e cíclicas sem remover datetime.
        """
        if self.df_base is None or self.df_base.empty:
            print("df_base vazio. Chame load_data() primeiro.")
            return self.df_base
        df = self.df_base.copy()
        if encode_method == 'label':
            le = LabelEncoder()
            s = df[encode_cols].astype(object)
            le.fit(s)
            df[encode_cols] = le.transform(s)
            # salva metadados incluindo o code do NaN
            self.encod_objects['label'] = {
                'encode_cols': encode_cols,
                'label_encoder': le,
            }
        elif encode_method == 'time_cycle':
            if encode_cols not in df.columns:
                print(f"Coluna {encode_cols} não encontrada para time_cycle.")
                self.df_base = df
                return df
            dt = pd.to_datetime(df[encode_cols], utc=True)
            # Mantém a coluna original e adiciona componentes discretos e cíclicos
            df['year'] = dt.dt.year
            df['month'] = dt.dt.month
            df['day'] = dt.dt.day
            df['hour'] = dt.dt.hour
            df['minute'] = dt.dt.minute
            current_year = time.localtime().tm_year
            df['year_sin'] = np.sin(2 * np.pi * df['year'] / max(current_year, 1))
            df['year_cos'] = np.cos(2 * np.pi * df['year'] / max(current_year, 1))
            df['month_sin'] = np.sin(2 * np.pi * df['month'] / 12)
            df['month_cos'] = np.cos(2 * np.pi * df['month'] / 12)
            df['day_sin'] = np.sin(2 * np.pi * df['day'] / 31)
            df['day_cos'] = np.cos(2 * np.pi * df['day'] / 31)
            df['hour_sin'] = np.sin(2 * np.pi * df['hour'] / 24)
            df['hour_cos'] = np.cos(2 * np.pi * df['hour'] / 24)
            df['minute_sin'] = np.sin(2 * np.pi * df['minute'] / 60)
            df['minute_cos'] = np.cos(2 * np.pi * df['minute'] / 60)
            self.encod_objects['time_cycle'] = {'encode_cols': encode_cols}
            self.feature_cols.extend(["year_sin", "year_cos",
                                                     "month_sin", "month_cos",
                                                     "day_sin", "day_cos",
                                                     "hour_sin", "hour_cos",
                                                     "minute_sin", "minute_cos"])
        else:
            print(f"encode_method '{encode_method}' não suportado.")
        self.df_base = df
        return self.df_base

    def decode(self, encode_method: str = 'label', target_col: Optional[str] = None) -> pd.DataFrame:
        """Reverte codificações suportadas (label, time_cycle)."""
        if self.df_base is None or self.df_base.empty:
            print("df_base vazio. Nada para decodificar.")
            return self.df_base
        df = self.df_base.copy()
        if encode_method == 'label':
            info = self.encod_objects.get('label')
            if not info:
                print("Nenhuma informação de label encoding salva.")
                return self.df_base
            col = info['encode_cols']
            le: LabelEncoder = info['label_encoder']
            placeholder = info.get('na_placeholder', '__NA__')
            try:
                inv = le.inverse_transform(df[col].astype(int))
                # mapeia placeholder de volta para NaN
                inv = pd.Series(inv).replace(placeholder, np.nan).values
                df[col] = inv
            except Exception as e:
                print(f"Falha ao decodificar label para coluna {col}: {e}")
        elif encode_method == 'time_cycle':
            if 'year' not in df.columns:
                print("Componentes de tempo ausentes para reconstrução.")
                return self.df_base
            tgt = target_col or 'decoded_datetime'
            def _recover_component(sin_col, cos_col, period, offset):
                if sin_col not in df.columns or cos_col not in df.columns:
                    return pd.Series([np.nan] * len(df))
                ang = np.arctan2(df[sin_col], df[cos_col])
                ang = (ang + 2 * np.pi) % (2 * np.pi)
                idx = np.round((ang / (2 * np.pi)) * period).astype('Int64') % period
                return idx + offset
            month = _recover_component('month_sin', 'month_cos', 12, 1)
            day = _recover_component('day_sin', 'day_cos', 31, 1)
            hour = _recover_component('hour_sin', 'hour_cos', 24, 0)
            minute = _recover_component('minute_sin', 'minute_cos', 60, 0)
            year = df['year'] if 'year' in df.columns else pd.Series([np.nan] * len(df))
            dt = pd.to_datetime({
                'year': year.astype('Int64'),
                'month': month.astype('Int64'),
                'day': day.astype('Int64'),
                'hour': hour.astype('Int64'),
                'minute': minute.astype('Int64'),
            }, errors='coerce', utc=True)
            df[tgt] = dt
        else:
            print(f"encode_method '{encode_method}' não suportado para decode.")
        self.df_base = df
        return self.df_base

    def normalize(self, value_cols: List[str], normalization_method: str = 'minmax') -> pd.DataFrame:
        """Normaliza colunas e atualiza self.df_base."""
        if self.df_base is None or self.df_base.empty:
            print("df_base vazio. Chame load_data() primeiro.")
            return self.df_base
        df = self.df_base.copy()
        scaler = MinMaxScaler() if normalization_method == 'minmax' else (
            StandardScaler() if normalization_method == 'standard' else None)
        if scaler is None:
            raise ValueError("normalization_method deve ser 'minmax' ou 'standard'")
        df[value_cols] = scaler.fit_transform(df[value_cols])
        self.norm_objects[normalization_method] = {'value_cols': value_cols, 'scaler': scaler}
        self.df_base = df
        return self.df_base

    def normalize_splits(self, value_cols: List[str], normalization_method: str = 'minmax') -> dict:
        """Normaliza os conjuntos de treino, validação e teste."""
        if not self.splits:
            print("Nenhum conjunto dividido encontrado.")
            return {}
        normalized_splits = {}
        for split_name, split_df in self.splits.items():
            self.df_base = split_df
            normalized_df = self.normalize(value_cols=value_cols, normalization_method=normalization_method)
            normalized_splits[split_name] = normalized_df
        self.splits = normalized_splits
        return normalized_splits

    def denormalize(self, normalization_method: str = 'minmax') -> pd.DataFrame:
        """Reverte normalização usando metadados salvos."""
        if self.df_base is None or self.df_base.empty:
            print("df_base vazio. Nada para denormalizar.")
            return self.df_base
        info = self.norm_objects.get(normalization_method)
        if not info:
            print(f"Nenhum scaler salvo para o método '{normalization_method}'.")
            return self.df_base
        cols: List[str] = info['value_cols']
        scaler = info['scaler']
        df = self.df_base.copy()
        try:
            df[cols] = scaler.inverse_transform(df[cols])
        except Exception as e:
            print(f"Falha ao denormalizar colunas {cols}: {e}")
            return self.df_base
        self.df_base = df
        return self.df_base

    def save_df_base(self, filename: Optional[str] = None, compression: Optional[str] = None, partition_by: Optional[List[str]] = None) -> Optional[str]:
        """Salva self.df_base em Parquet dentro de data_dir/{model_name}."""
        if self.df_base is None or self.df_base.empty:
            print("df_base vazio. Nada para salvar.")
            return None
        comp = compression
        if comp is None:
            try:
                comp = PARQUET_COMPRESSION
            except NameError:
                comp = 'zstd'
        filename = "raw_dataset.parquet"
        out_path = os.path.join(self.save_dir, filename)
        df = self.df_base.copy()
        if 'datetime' in df.columns:
            df['datetime'] = pd.to_datetime(df['datetime'], utc=True)
        try:
            if partition_by:
                df.to_parquet(out_path, engine='pyarrow', compression=comp, index=False, partition_cols=partition_by)
            else:
                df.to_parquet(out_path, engine='pyarrow', compression=comp, index=False)
            print(f"[SALVO] df_base: {len(df):,} linhas → {out_path}")
            return out_path
        except Exception as e:
            print(f"Falha ao salvar df_base em {out_path}: {e}")
            return None
    
    def split_train_val_test(self, train_size: float = 0.7, val_size: float = 0.15, test_size: float = 0.15, time_col: str = 'datetime') -> Optional[dict]:
        """Divide df_base em conjuntos de treino, validação e teste com base em time_col."""
        if self.df_base is None or self.df_base.empty:
            print("df_base vazio. Nada para dividir.")
            return None
        if not np.isclose(train_size + val_size + test_size, 1.0):
            print("train_size, val_size e test_size devem somar 1.0")
            return None
        df = self.df_base.copy()
        if time_col not in df.columns:
            print(f"Coluna de tempo '{time_col}' não encontrada em df_base.")
            return None
        df = df.sort_values(time_col).reset_index(drop=True)
        n = len(df)
        train_end = int(n * train_size)
        val_end = train_end + int(n * val_size)
        splits = {
            'train': df.iloc[:train_end].reset_index(drop=True),
            'val': df.iloc[train_end:val_end].reset_index(drop=True),
            'test': df.iloc[val_end:].reset_index(drop=True),
        }
        for split_name, split_df in splits.items():
            print(f"[DIVIDIDO] {split_name}: {len(split_df):,} linhas")
        self.splits = splits
        return splits

## Preprocessamento do Modelo Linear

Esse modelo deve será contruido a partir de lags e leads passados como parâmetros na função, resultando na contrução de novas colunas lead lag, assim gerando uma flat matrix 2D que será usada no modelo linear

Observação importante: lag e lead são inteiros e representam o máximo de passos; o pipeline expande para intervalos 1..N automaticamente. Por exemplo, lag=96 gera features com defasagens de 1 a 96; lead=96 gera alvos de 1 a 96.

Os arquivos do modelo serão salvos em Parquet, já que o modelo linear será construído usando TensorFlow (carregado via Parquet→NumPy→tf.data)

No caso o Preprocessador do modelo linear será igual ao pré-processador do MLP 

In [20]:
class LinearPreprocessor(Preprocessor):
    """Pré-processador linear: gera matriz flat (lags/leads) e exporta apenas Parquet."""

    def build_flat_matrix(
        self,
        value_cols: Optional[List[str]] = None,
        target_cols: Optional[List[str]] = None,
        lags: Optional[int] = None,
        leads: Optional[int] = None,
        reduced_dim: Optional[int] = None,
        mask_value: float = 0.0,
        dropna: bool = True,
        group_cols: Optional[List[str]] = None,
        time_col: str = "datetime",
    ) -> pd.DataFrame:
        import pandas as pd
        import numpy as np

        if self.df_base is None or self.df_base.empty:
            print("df_base vazio. Chame load_data() primeiro.")
            return self.df_base

        df = self.df_base.copy()
        feats = value_cols or self.feature_cols
        tgts = target_cols or self.target_cols
        if not feats:
            raise ValueError("Nenhuma coluna de feature informada.")
        if not tgts:
            raise ValueError("Nenhum target informado.")

        group_cols = group_cols or [c for c in ["country"] if c in df.columns]
        if time_col not in df.columns:
            raise ValueError(f"Coluna temporal '{time_col}' não encontrada no DataFrame.")

        # Ordena
        sort_cols = (group_cols or []) + [time_col]
        df = df.sort_values(sort_cols).reset_index(drop=True)

        if group_cols:
            df["_group_id"] = df[group_cols].astype(str).agg("_".join, axis=1)
        else:
            df["_group_id"] = "global"

        lag_steps = list(range(1, (lags or self.lag or 0) + 1))
        lead_steps = list(range(1, (leads or self.lead or 0) + 1))
        new_cols = []

        # Determina redução
        if reduced_dim is not None and reduced_dim < len(lag_steps):
            active_lags = lag_steps[:reduced_dim]
            padded_lags = lag_steps[reduced_dim:]
            print(f"[ℹ️] Reduzindo lags: usando {len(active_lags)} e mascarando {len(padded_lags)} restantes com {mask_value}")
        else:
            active_lags = lag_steps
            padded_lags = []

        # ---- Lags ----
        for col in feats:
            if col not in df.columns:
                print(f"[WARN] Coluna de feature '{col}' não encontrada.")
                continue
            # Lags ativos
            for k in active_lags:
                cname = f"{col}_lag{k}"
                df[cname] = df.groupby("_group_id", group_keys=False, sort=False)[col].shift(k)
                new_cols.append(cname)
            # Lags mascarados (padding)
            for k in padded_lags:
                cname = f"{col}_lag{k}"
                df[cname] = mask_value
                new_cols.append(cname)

        # ---- Leads ----
        for tgt in tgts:
            if tgt in df.columns:
                for k in lead_steps:
                    cname = f"{tgt}_lead{k}"
                    df[cname] = df.groupby("_group_id", group_keys=False, sort=False)[tgt].shift(-k)
                    new_cols.append(cname)
            else:
                print(f"[WARN] Target '{tgt}' não encontrado. Ignorando leads.")

        # ---- Drop NA ----
        if dropna and new_cols:
            df = df.dropna(subset=[c for c in new_cols if mask_value not in df[c].unique()]).reset_index(drop=True)

        df.drop(columns=["_group_id"], inplace=True, errors="ignore")

        # Atualiza atributos
        self.df_base = df
        self.feature_cols.extend([c for c in new_cols if "_lag" in c and c not in self.feature_cols])
        self.target_cols.extend([c for c in new_cols if "_lead" in c and c not in self.target_cols])

        return self.df_base

    def build_flat_matrices_splits(self, *args, **kwargs) -> Optional[dict]:
        """Constrói matrizes flat para cada split (train/val/test)."""
        if not self.splits:
            print("Nenhum conjunto dividido encontrado.")
            return None
        built_splits = {}
        for split_name, split_df in self.splits.items():
            self.df_base = split_df
            built_df = self.build_flat_matrix(*args, **kwargs)
            built_splits[split_name] = built_df
        self.splits = built_splits
        return built_splits


## Preprocessamento do Modelo LSTM

O preprocessador do LSTM deve ser capaz de gerar tensores de dimensão 3, no seguinte formato (n_batch, seq_len, features) e (n_batch, seq_len, features)  para alimentação do modelo e validação das métricas do modelo

Os arquivos do modelo serão salvos em Parquet (colunas X e Y como listas fixas), e carregados via Parquet→NumPy→tf.data

In [21]:
import os, json, re
import numpy as np
import pandas as pd
import tensorflow as tf
from typing import List, Optional, Dict, Any


class LSTMPreprocessor(Preprocessor):
    """Pré-processador sequencial para LSTM: gera janelas 3D (N, seq_len, features) e Y (N, lead, targets)."""

    # =====================================================
    # BUILD MATRIX
    # =====================================================
    def build_sequence_matrix(
        self,
        value_cols: Optional[List[str]] = None,
        target_cols: Optional[List[str]] = None,
        seq_len: Optional[int] = None,
        lead: Optional[int] = None,
        group_cols: Optional[List[str]] = None,
        time_col: str = "datetime",
    ) -> Dict[str, np.ndarray]:
        """
        Constrói tensores X (entradas) e Y (alvos) para modelo LSTM multivariado.
        Agora suporta múltiplos passos de previsão (lead > 1).
        """
        if self.df_base is None or self.df_base.empty:
            raise ValueError("df_base vazio. Chame load_data() primeiro.")

        df = self.df_base.copy()
        feats = value_cols or self.feature_cols
        tgts = target_cols or self.target_cols
        if not feats:
            raise ValueError("Nenhuma coluna de feature informada.")
        if not tgts:
            raise ValueError("Nenhum target informado.")
        if time_col not in df.columns:
            raise ValueError(f"Coluna temporal '{time_col}' não encontrada.")

        group_cols = group_cols or [c for c in ["country"] if c in df.columns]
        sort_cols = (group_cols or []) + [time_col]
        df = df.sort_values(sort_cols).reset_index(drop=True)

        if group_cols:
            df["_group_id"] = df[group_cols].astype(str).agg("_".join, axis=1)
        else:
            df["_group_id"] = "global"

        seq_len = seq_len or getattr(self, "seq_len", 24)
        lead = lead or getattr(self, "lead", 1)

        X_list, Y_list = [], []
        for gid, g in df.groupby("_group_id", sort=False):
            g = g.reset_index(drop=True)
            if len(g) < seq_len + lead:
                continue

            X_src = g[feats].to_numpy(np.float32)
            Y_src = g[tgts].to_numpy(np.float32)

            # cria janelas deslizantes
            for i in range(len(g) - seq_len - lead + 1):
                x_win = X_src[i : i + seq_len]
                y_seq = Y_src[i + seq_len : i + seq_len + lead]  # <--- multi-step
                X_list.append(x_win)
                Y_list.append(y_seq)

        if not X_list:
            print("[WARN] Nenhuma janela gerada.")
            return {}

        X = np.stack(X_list)  # (N, seq_len, x_dim)
        Y = np.stack(Y_list)  # (N, lead, y_dim)
        print(f"[JANELAS] X={X.shape}, Y={Y.shape}, seq_len={seq_len}, lead={lead}")
        self._seq_data = dict(
            X=X, Y=Y, seq_len=seq_len, lead=lead, x_dim=X.shape[-1], y_dim=Y.shape[-1]
        )
        return self._seq_data

    # =====================================================
    # SAVE PARQUET (streaming por splits)
    # =====================================================
    def save_splits_parquet(self, basename: str = "lstm_dataset", chunk_windows: int = 10_000, verbose: bool = True) -> Dict[str, Any]:
        """
        Gera janelas (seq_len -> lead) diretamente a partir de self.splits e salva Parquet por split usando
        pyarrow ParquetWriter com FixedSizeList para evitar picos de memória.
        """
        import pyarrow as pa
        import pyarrow.parquet as pq

        def _uniq(seq):
            seen = set()
            out = []
            for x in seq:
                if x not in seen:
                    out.append(x)
                    seen.add(x)
            return out

        if not getattr(self, 'splits', None):
            raise ValueError("Nenhum split encontrado. Execute split_train_val_test() antes.")

        # Resolve seq_len/lead a partir de atributos do preproc
        seq_len_attr = getattr(self, 'seq_len', None)
        if seq_len_attr is None:
            seq_len_attr = getattr(self, 'lag', None)
        if seq_len_attr is None:
            raise AttributeError("Pré-processador não possui atributos 'seq_len' ou 'lag'.")
        seq_len = int(seq_len_attr)

        lead_attr = getattr(self, 'lead', None)
        if lead_attr is None:
            lead_attr = getattr(self, 'horizon', None)
        if lead_attr is None:
            raise AttributeError("Pré-processador não possui atributo 'lead' (ou 'horizon').")
        lead = int(lead_attr)

        out: Dict[str, Any] = {}
        for split_name, df in self.splits.items():
            # Seleção e de-duplicação de colunas
            fraw = [c for c in self.feature_cols if c in df.columns]
            traw = [c for c in self.target_cols if c in df.columns]
            tcols = _uniq(traw)
            fcols = [c for c in _uniq(fraw) if c not in set(tcols)]

            x_dim = len(fcols)
            y_dim = len(tcols)
            Xf = df[fcols].to_numpy(dtype=np.float32)
            Yf = df[tcols].to_numpy(dtype=np.float32)
            n = len(df)
            n_windows = max(0, n - seq_len - lead + 1)

            if verbose:
                print(f"[LSTM:{split_name}] linhas={n:,}  windows={n_windows:,}  X=({seq_len},{x_dim})  Y=({lead},{y_dim})  chunk={chunk_windows:,}")

            os.makedirs(self.data_dir, exist_ok=True)
            pq_path = os.path.join(self.data_dir, f"{basename}_{split_name}.parquet")

            fx = seq_len * x_dim
            fy = lead * y_dim
            schema = pa.schema([
                pa.field("start_idx", pa.int32()),
                pa.field("X", pa.list_(pa.float32(), fx)),
                pa.field("Y", pa.list_(pa.float32(), fy)),
            ])
            writer = pq.ParquetWriter(pq_path, schema, compression="snappy")

            try:
                for s in range(0, n_windows, chunk_windows):
                    e = min(s + chunk_windows, n_windows)
                    csize = e - s
                    X_chunk = np.empty((csize, fx), dtype=np.float32)
                    Y_chunk = np.empty((csize, fy), dtype=np.float32)
                    starts = np.arange(s, e, dtype=np.int32)
                    for i, start in enumerate(starts):
                        xw = Xf[start:start+seq_len]
                        yw = Yf[start+seq_len:start+seq_len+lead]
                        X_chunk[i, :] = xw.reshape(-1)
                        Y_chunk[i, :] = yw.reshape(-1)
                    X_values = pa.array(X_chunk.reshape(-1), type=pa.float32())
                    Y_values = pa.array(Y_chunk.reshape(-1), type=pa.float32())
                    X_arr = pa.FixedSizeListArray.from_arrays(X_values, fx)
                    Y_arr = pa.FixedSizeListArray.from_arrays(Y_values, fy)
                    table = pa.Table.from_arrays([pa.array(starts), X_arr, Y_arr], names=["start_idx", "X", "Y"])
                    writer.write_table(table)
            finally:
                writer.close()

            meta = {
                "seq_len": seq_len,
                "lead": lead,
                "x_dim": x_dim,
                "y_dim": y_dim,
                "feature_cols": fcols,
                "target_cols": tcols,
                "parquet_path": pq_path,
                "basename": f"{basename}_{split_name}",
            }
            meta_path = os.path.join(self.data_dir, f"{basename}_{split_name}.meta.json")
            with open(meta_path, "w", encoding="utf-8") as f:
                json.dump(meta, f, ensure_ascii=False, indent=2)
            out[split_name] = {"path": pq_path, "meta": meta}
            if verbose:
                print(f"[💾] Parquet LSTM salvo: {pq_path}")
        return out


## Preprocessamento do Modelo TFT (PyTorch)

O preprocessador do LSTM deve ser capaz de gerar tensores de dimensão 3, no seguinte formato (n_batch, seq_len, features) e (n_batch, seq_len, features)  para alimentação do modelo e validação das métricas do modelo

Os artefatos de dados serão salvos em Parquet (Keras e TFT).

In [22]:
import os
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from pytorch_forecasting import TimeSeriesDataSet
from pytorch_forecasting.models import TemporalFusionTransformer
from lightning.pytorch import Trainer, seed_everything
from pytorch_lightning.callbacks import EarlyStopping, LearningRateMonitor, ModelCheckpoint
from typing import List, Optional


class TFTPreprocessor(Preprocessor):
    """
    Preprocessador específico para o modelo Temporal Fusion Transformer (PyTorch Forecasting).
    Herdando de Preprocessor, apenas adiciona a etapa final de estruturação e salvamento
    dos splits no formato compatível com o PyTorch Forecasting.
    """

    def __init__(
        self,
        data_dir: str,
        model_name: str,
        feature_cols: List[str],
        target_cols: List[str],
        country_list: List[str],
        seq_len: int,
        lead: int,
    ):
        # Corrigido: alinhar com assinatura de Preprocessor
        super().__init__(
            lag=seq_len,
            lead=lead,
            country_list=country_list,
            model_name=model_name,
            data_dir=data_dir,
            feature_cols=feature_cols,
            target_cols=target_cols,
        )
        self.seq_len = seq_len
        self.lead = lead


    def build_tft_parquets(
        self,
        group_cols: Optional[List[str]] = ["country"],
        time_col: str = "datetime",
        dropna: bool = True,
    ):
        """
        Estrutura os splits existentes (já criados na classe-base) para uso no TFT e salva em parquet.
        Simples e direto:
        - Ordena por (group_cols + time_col)
        - Opcionalmente remove nulos nas colunas críticas [time_col] + group_cols + target_cols
        - Define '_group_id' e calcula 'time_idx' por grupo via cumcount() (0..N-1 por série)
        - Salva parquet por split
        """
        if not hasattr(self, "splits") or not self.splits:
            raise ValueError("Os splits ainda não foram gerados. Execute split_train_val_test() primeiro.")

        for name, df in self.splits.items():
            df = df.copy()
            # tipos e ordenação
            df[time_col] = pd.to_datetime(df[time_col], utc=True)
            sort_cols = (group_cols or []) + [time_col]
            df = df.sort_values(sort_cols).reset_index(drop=True)

            # drop nulos básico
            if dropna:
                subset_cols = ([time_col] if time_col else []) + (group_cols or []) + (self.target_cols or [])
                present = [c for c in subset_cols if c in df.columns]
                before = len(df)
                df = df.dropna(subset=present).reset_index(drop=True)
                if before - len(df) > 0:
                    print(f"🧹 Drop NA ({name}): {before - len(df)} linhas removidas nas colunas {present}.")

            # id de grupo e time_idx por grupo
            if group_cols:
                df["_group_id"] = df[group_cols].astype(str).agg("_".join, axis=1)
            else:
                df["_group_id"] = "global"

            # contador sequencial por grupo (não global)
            df["time_idx"] = df.groupby("_group_id").cumcount().astype("int64")

            # salvar parquet
            path = os.path.join(self.data_dir, f"tft_dataset_{name}.parquet")
            df.to_parquet(path, index=False)
            print(f"💾 Split '{name}' salvo em {path} ({df.shape[0]} linhas, grupos={df['_group_id'].nunique()}, max local time_idx={df.groupby('_group_id')['time_idx'].max().max()}).")


    def load_tft_dataset(
        self,
        split_name: str,
        target_col: str,
        known_reals: Optional[List[str]] = None,
        return_df: bool = False,
    ):
        """
        Carrega o parquet salvo como DataFrame ou cria um TimeSeriesDataSet compatível com o TFT PyTorch.

        Args:
            split_name: 'train' | 'val' | 'test' (parte do nome do arquivo parquet gerado)
            target_col: coluna alvo principal (string)
            known_reals: lista de features conhecidas no tempo (overrides self.feature_cols quando fornecida)
            return_df: se True retorna o DataFrame bruto em vez do TimeSeriesDataSet

        Retorna:
            DataFrame (quando return_df=True) ou TimeSeriesDataSet
        """
        path = os.path.join(self.data_dir, f"tft_dataset_{split_name}.parquet")
        if not os.path.exists(path):
            raise FileNotFoundError(f"Arquivo não encontrado: {path}")

        df = pd.read_parquet(path)

        if return_df:
            print(f"📥 Parquet '{split_name}' carregado ({len(df)} linhas) — retornando DataFrame.")
            return df

        # determina known/unknown reals
        known_reals = known_reals or [c for c in (self.feature_cols or []) if c not in (self.target_cols or [])]

        ds = TimeSeriesDataSet(
            df,
            time_idx="time_idx",
            target=target_col,
            group_ids=["_group_id"],
            max_encoder_length=self.seq_len,
            max_prediction_length=self.lead,
            time_varying_known_reals=known_reals,
            time_varying_unknown_reals=self.target_cols,
            add_relative_time_idx=True,
            add_target_scales=True,
            add_encoder_length=True,
        )
        print(f"📦 TimeSeriesDataSet ({split_name}) criado com {len(df)} amostras.")
        return ds

# Capítulo 2 — Construção dos Modelos

A seguir, definimos construtores simples e eficientes para cada modelo (Linear, LSTM, TFT e TimesFM),
prontos para uso em rotinas de otimização de hiperparâmetros (por exemplo, Optuna). Cada construtor
recebe um dicionário de parâmetros (`params`) e retorna um modelo compilado.

## Construção do Modelo Linear/MLP

Objetivo: Criar um regressor simples (MLP), com capacidade de redução para um modelo apenas lienar - pela exclusão da camada de ativação - para prever `target_cols` a partir de `feature_cols`.

Contrato rápido:
- Entrada: vetor de tamanho `x_dim` (número de features)
- Saída: vetor de tamanho `y_dim` (número de targets)
- Parâmetros (exemplos): hidden_units, activation, dropout, lr, l2

In [23]:
# Linear/MLP model + loaders (Parquet only)
from typing import Dict, Any
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers


def build_linear_model(x_dim: int, y_dim: int, params: Dict[str, Any], linear: bool = False) -> keras.Model:
    """
    Constrói um modelo linear puro ou um MLP, dependendo do argumento `linear`.

    params:
      - hidden_units: List[int] (apenas usado se linear=False)
      - activation: str (apenas usado se linear=False)
      - dropout: float (0..1)
      - l2: float (regularização L2)
      - lr: float (learning rate)
    """
    hidden_units = params.get('hidden_units', [128, 64])
    activation = params.get('activation', 'relu')
    dropout = float(params.get('dropout', 0.0))
    l2 = float(params.get('l2', 0.0))
    lr = float(params.get('lr', 1e-3))
    mask_value = params.get('mask_value', -999.0)

    inputs = keras.Input(shape=(x_dim,), name='features')

    x = layers.Masking(mask_value=mask_value, name='masking')(inputs)

    if linear:
        # Modelo puramente linear (sem ativação)
        outputs = layers.Dense(
            y_dim,
            activation=None,
            kernel_regularizer=keras.regularizers.l2(l2),
            name='linear_output'
        )(inputs)
        model = keras.Model(inputs, outputs, name='linear_model_true')

    else:
        # Modelo MLP (não linear)
        x = inputs
        for i, units in enumerate(hidden_units):
            x = layers.Dense(
                units,
                activation=activation,
                kernel_regularizer=keras.regularizers.l2(l2),
                name=f'dense_{i}'
            )(x)
            if dropout > 0:
                x = layers.Dropout(dropout, name=f'dropout_{i}')(x)
        outputs = layers.Dense(y_dim, name='targets')(x)
        model = keras.Model(inputs, outputs, name='mlp_model')

    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=lr),
        loss='mse',
        metrics=['mae']
    )
    return model


# -------- Parquet helpers for Linear/MLP --------

def save_linear_splits_parquet(preproc, basename: str = "linear_dataset", row_group_size: int = 200_000, verbose: bool = True) -> dict:
    """
    Salva cada split (train/val/test) como Parquet + meta.json com x_dim/y_dim/colunas.
    De-duplica listas de colunas e evita sobreposição entre features e targets.
    Usa pyarrow ParquetWriter com row groups (chunks) para reduzir picos de memória.
    """
    import json, os
    import pandas as pd
    import pyarrow as pa
    import pyarrow.parquet as pq

    def _uniq(seq):
        seen = set()
        out = []
        for x in seq:
            if x not in seen:
                out.append(x)
                seen.add(x)
        return out

    out = {}
    for split_name, df in (getattr(preproc, 'splits', {}) or {}).items():
        path = os.path.join(preproc.data_dir, f"{basename}_{split_name}.parquet")
        num_cols = df.select_dtypes(include=["number", "bool"]).columns.tolist()
        fraw = [c for c in preproc.feature_cols if c in num_cols]
        traw = [c for c in preproc.target_cols if c in num_cols]
        tcols = _uniq(traw)
        fcols = [c for c in _uniq(fraw) if c not in set(tcols)]
        combined_cols = fcols + tcols

        if verbose:
            print(f"[Linear:{split_name}] linhas={len(df):,}  X={len(fcols)}  Y={len(tcols)}  escrevendo em chunks de {row_group_size:,}…")

        writer = None
        try:
            n = len(df)
            for start in range(0, n, row_group_size):
                end = min(start + row_group_size, n)
                chunk = df.iloc[start:end][combined_cols]
                table = pa.Table.from_pandas(chunk, preserve_index=False)
                if writer is None:
                    writer = pq.ParquetWriter(path, table.schema, compression="snappy")
                writer.write_table(table)
            if writer is not None:
                writer.close()
        finally:
            if writer is not None:
                try:
                    writer.close()
                except Exception:
                    pass

        meta = {
            "x_dim": int(len(fcols)),
            "y_dim": int(len(tcols)),
            "feature_cols": fcols,
            "target_cols": tcols,
            "parquet_path": path,
            "basename": f"{basename}_{split_name}"
        }
        with open(os.path.join(preproc.data_dir, f"{basename}_{split_name}.meta.json"), "w", encoding="utf-8") as f:
            json.dump(meta, f, ensure_ascii=False, indent=2)
        out[split_name] = {"path": path, "meta": meta}
        if verbose:
            print(f"[💾] Parquet salvo: {path}")
    return out


def load_linear_parquet_dataset(data_dir: str, split: str, batch_size: int = 256, shuffle: bool = True):
    """
    Carrega Parquet para NumPy e monta tf.data a partir de memória (rápido).
    """
    import os, json
    import pandas as pd
    import tensorflow as tf

    meta_path = os.path.join(data_dir, f"linear_dataset_{split}.meta.json")
    if not os.path.exists(meta_path):
        raise FileNotFoundError(f"Meta JSON não encontrado: {meta_path}")
    with open(meta_path, "r", encoding="utf-8") as f:
        meta = json.load(f)
    df = pd.read_parquet(meta["parquet_path"])
    X = df[meta["feature_cols"]].to_numpy("float32", copy=False)
    Y = df[meta["target_cols"]].to_numpy("float32", copy=False)
    ds = tf.data.Dataset.from_tensor_slices((X, Y))
    if shuffle:
        ds = ds.shuffle(min(len(df), 10000), seed=42, reshuffle_each_iteration=True)
    ds = ds.batch(batch_size).prefetch(tf.data.AUTOTUNE)
    return ds, meta


## Construção do Modelo LSTM

Objetivo: um regressor denso simples (MLP) para prever `target_cols` a partir de `feature_cols`.

Contrato rápido:
- Entrada: vetor de tamanho `x_dim` (número de features)
- Saída: vetor de tamanho `y_dim` (número de targets)
- Parâmetros (exemplos): hidden_units, activation, dropout, lr, l2

In [24]:
from typing import Dict, Any
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

from typing import Dict, Any, Tuple
import numpy as np
import pandas as pd
import os, json
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers



def build_lstm_model(seq_len: int, x_dim: int, y_dim: int, lead: int, params: Dict[str, Any]) -> keras.Model:
    """
    LSTM seq2seq simples para previsão multi-passo:
      encoder LSTM -> RepeatVector(lead) -> decoder LSTM -> TimeDistributed(Dense(y_dim)).

    Parâmetros aceitos (em params):
      - lstm_units: int OU lista/tupla de int
          * int: usa o mesmo valor no encoder e decoder
          * lista/tupla: usa primeiro para encoder e último para decoder
      - dropout: float (0..1)
      - recurrent_dropout ou rec_dropout: float (0..1)
      - lr: float (learning rate)
      - bidirectional: bool (usa Bidirectional no encoder)
    """
    # lstm_units flexível: int ou lista/tupla
    lu = params.get('lstm_units', 128)
    if isinstance(lu, (list, tuple)):
        try:
            enc_units = int(lu[0])
            dec_units = int(lu[-1])
        except Exception:
            raise ValueError(f"lstm_units inválido (esperado int ou lista de ints), recebido: {lu}")
    else:
        enc_units = dec_units = int(lu)

    dropout = float(params.get('dropout', 0.0))
    rdrop = float(params.get('recurrent_dropout', params.get('rec_dropout', 0.0)))
    lr = float(params.get('lr', 1e-3))
    bidir = bool(params.get('bidirectional', False))

    inputs = keras.Input(shape=(seq_len, x_dim), name="X")
    if bidir:
        enc = layers.Bidirectional(
            layers.LSTM(enc_units, dropout=dropout, recurrent_dropout=rdrop, return_sequences=False),
            name="enc_bi"
        )(inputs)
    else:
        enc = layers.LSTM(enc_units, dropout=dropout, recurrent_dropout=rdrop, return_sequences=False, name="enc")(inputs)

    rep = layers.RepeatVector(lead, name="repeat_lead")(enc)
    dec = layers.LSTM(dec_units, dropout=dropout, recurrent_dropout=rdrop, return_sequences=True, name="dec")(rep)
    outputs = layers.TimeDistributed(layers.Dense(y_dim), name="td_out")(dec)

    model = keras.Model(inputs, outputs, name="lstm_seq2seq")
    model.compile(optimizer=keras.optimizers.Adam(learning_rate=lr), loss="mse", metrics=["mae"]) 
    return model

def load_lstm_parquet_dataset(data_dir: str, split: str, batch_size: int = 64, shuffle: bool = True):
    """
    Lê Parquet com colunas X(list<float>), Y(list<float>) e reconstrói tensores:
      X -> (seq_len, x_dim), Y -> (lead, y_dim)
    Retorna tf.data.Dataset[(X, Y)] e meta.
    """
    meta_path = os.path.join(data_dir, f"lstm_dataset_{split}.meta.json")
    if not os.path.exists(meta_path):
        raise FileNotFoundError(f"Meta JSON não encontrado: {meta_path}")
    with open(meta_path, "r", encoding="utf-8") as f:
        meta = json.load(f)
    pq = meta["parquet_path"]
    df = pd.read_parquet(pq)

    seq_len = int(meta["seq_len"]) ; lead = int(meta["lead"]) ; x_dim = int(meta["x_dim"]) ; y_dim = int(meta["y_dim"]) 

    # Converte listas -> numpy e reshape por janela
    X_list = df["X"].to_list()
    Y_list = df["Y"].to_list()
    X = np.asarray(X_list, dtype=np.float32).reshape((-1, seq_len, x_dim))
    Y = np.asarray(Y_list, dtype=np.float32).reshape((-1, lead, y_dim))

    ds = tf.data.Dataset.from_tensor_slices((X, Y))
    if shuffle:
        ds = ds.shuffle(min(len(df), 10000), seed=42, reshuffle_each_iteration=True)
    ds = ds.batch(batch_size).prefetch(tf.data.AUTOTUNE)
    return ds, meta




## Construção do Modelo TFT (Temporal Fusion Transformer)

**Objetivo:** prever `target_cols` a partir de `feature_cols` utilizando a implementação oficial `keras_tft`,  
que integra **seleção de variáveis dinâmicas**, **blocos LSTM**, **atenção temporal multi-cabeças** e **gating residual** em um único modelo interpretable.

**Contrato rápido:**
- **Entrada:** `tf.data.Dataset` com tensores no formato `(batch, seq_len, x_dim)`  
- **Saída:** tensor contínuo de tamanho `y_dim` *(ou `dec_len × y_dim` para horizontes múltiplos)*

**Parâmetros (exemplos):**  
`hidden_size` (tamanho interno das camadas GRN) · `lstm_layers` · `num_heads` (atenção) · `dropout` · `learning_rate` · `output_size` · `seq_len`

**Componentes internos (`keras_tft`):**  
Variable Selection Network → LSTM Encoder/Decoder → Multi-Head Temporal Attention → Gated Residual Network → Camada de projeção final

**Compatibilidade:**  
Totalmente compatível com o pipeline atual em Parquet do LSTM, recebendo o mesmo formato de dados  
(`(batch, seq_len, features)`), permitindo substituição direta do modelo sem alterar o pré-processamento.



In [25]:
from typing import Dict, Any
import torch
from torch import nn
from pytorch_forecasting.models import TemporalFusionTransformer
from pytorch_forecasting.metrics import QuantileLoss

def build_tft_model(
    params: Dict[str, Any]
):
    """
    Constrói um Temporal Fusion Transformer (TFT) com parâmetros configuráveis.

    Args:
        x_dim: número de features de entrada
        y_dim: número de targets
        seq_len: tamanho da sequência temporal
        params: dicionário de hiperparâmetros (hidden_size, dropout, lstm_layers, etc.)
        max_encoder_length: tamanho da janela passada (encoder)
        max_prediction_length: tamanho do horizonte de previsão (decoder)
    """

    from pytorch_forecasting.models.temporal_fusion_transformer.tuning import optimize_hyperparameters

    hidden_size = int(params.get("hidden_size", 64))
    dropout = float(params.get("dropout", 0.1))
    lstm_layers = int(params.get("lstm_layers", 1))
    attention_head_size = int(params.get("num_heads", 4))
    lr = float(params.get("lr", 1e-3))

    model = TemporalFusionTransformer.from_dataset(
        params["dataset"],  # dataset preparado via TimeSeriesDataSet
        learning_rate=lr,
        hidden_size=hidden_size,
        dropout=dropout,
        lstm_layers=lstm_layers,
        attention_head_size=attention_head_size,
        loss=QuantileLoss([0.5]),
        log_interval=10,
        log_val_interval=1
    )

    return model


# Capítulo 3 - Contrução da Pipelines de dados dos modelos

A função de pipeline organiza o fluxo de dados para, de forma mais concisa e organizada, treinar o modelo, sendo capaz de mostrar a progressão das perdas a medida que as épocas de treinamento passam - Esse display esté disponível no notebook "Resultados"

O resultado da pipeline é um gráfico com a evolução de todas as métricas e o salvamento do modelo treinado dentro da pasta ./modelo/{Nome_Problema}/{Nome_Modelo}

Assim podendo ser facilmente reutilizado futuramente para um notebook comparativo

## Pipeline dos Modelos Lineares

Pipeline de preprocessamento e de treinamento

In [26]:
# Utilitário para salvar modelos Keras de forma padronizada
# Uso: save_model(model, path="./modelos/<problema>/<nome>") → salva "<nome>.keras" e um meta.json

def save_model(model, path: str, format: str | None = None, include_optimizer: bool = True) -> str:
    """
    Salva um modelo Keras em formato padronizado.

    Regras:
    - Se `path` terminar com .keras ou .h5, salva exatamente nesse arquivo.
    - Se `format == 'savedmodel'`, salva no diretório indicado (SavedModel).
    - Caso contrário, adiciona sufixo .keras a `path` (arquivo único Keras v3).

    Retorna o caminho final salvo (arquivo ou diretório) e grava um meta.json ao lado.
    """
    import os, json, datetime

    # Inferência de formato por extensão
    ext = None
    lower = path.lower()
    if lower.endswith(".keras"):
        ext = "keras"
    elif lower.endswith(".h5") or lower.endswith(".hdf5"):
        ext = "h5"

    # Normalização de destino
    if format == "savedmodel":
        # Diretório SavedModel
        save_dir = path
        os.makedirs(save_dir, exist_ok=True)
        model.save(save_dir, include_optimizer=include_optimizer)
        meta_path = os.path.join(save_dir, "model.meta.json")
        final_path = save_dir
    else:
        if ext is None:
            # Força arquivo .keras por padrão
            path = f"{path}.keras"
            ext = "keras"
        # Cria diretório pai
        parent = os.path.dirname(path) or "."
        os.makedirs(parent, exist_ok=True)
        # Salva arquivo único
        model.save(path, include_optimizer=include_optimizer)
        meta_path = f"{path}.meta.json"
        final_path = path

    # Meta básico ao lado do artefato
    try:
        meta = {
            "saved_at": datetime.datetime.utcnow().isoformat() + "Z",
            "keras_version": getattr(model, "keras_version", None),
            "model_name": getattr(model, "name", None),
            "trainable_params": int(getattr(model, "count_params", lambda: 0)()),
            "format": "savedmodel" if format == "savedmodel" else ext,
        }
        with open(meta_path, "w", encoding="utf-8") as f:
            json.dump(meta, f, ensure_ascii=False, indent=2)
    except Exception as e:
        print(f"[WARN] Falha ao escrever meta.json: {e}")

    print(f"[💾] Modelo salvo em: {final_path}")
    return final_path


In [27]:
import os
import matplotlib.pyplot as plt
from typing import List, Tuple, Dict, Any, Optional
from tensorflow import keras
from tensorflow.keras.callbacks import EarlyStopping as TFEarlyStopping, ReduceLROnPlateau as TFReduceLROnPlateau


def linear_preproccess_pipeline(
    data_dir: str,
    country_list: List[str],
    feature_cols: List[str],
    target_cols: List[str],
    lag: int,
    lead: int,
    value_cols: List[str],
    reduced_dim: Optional[int] = None,
    mask_value: Optional[float] = -999.0,
) -> Tuple[LinearPreprocessor, Dict[str, keras.Model]]:
    """
    Pipeline completa de pré-processamento (Parquet only) e treinamento de 3 modelos
    (simple, medium, deep) para comparação direta de desempenho.
    """

    # ----------------------------
    # Pré-processamento
    # ----------------------------
    preproc = LinearPreprocessor(
        data_dir=data_dir,
        model_name='linear_model',
        feature_cols=feature_cols,
        target_cols=target_cols,
        lag=lag,
        lead=lead,
        country_list=country_list
    )

    preproc.load_data()
    preproc.encode(encode_cols='datetime', encode_method='time_cycle')
    preproc.encode(encode_cols='country', encode_method='label')
    preproc.split_train_val_test(train_size=0.8, val_size=0.1, test_size=0.1, time_col='datetime')
    preproc.normalize_splits(value_cols=value_cols, normalization_method='minmax')
    preproc.build_flat_matrices_splits(
        value_cols=value_cols,
        target_cols=target_cols,
        dropna=True,
        group_cols=['country'],
        time_col='datetime',
        reduced_dim=reduced_dim,
        mask_value=mask_value
    )
    # Parquet only
    save_linear_splits_parquet(preproc, basename='linear_dataset')
    print("✅ Pré-processamento linear concluído (Parquet).")

    return preproc


def linear_train_pipeline(
    problem_name: str,
    data_dir: str,
    batch_size: int = 128,
    configs: Dict[str, Dict[str, Any]] = None
):
    # ----------------------------
    # Parquet datasets
    # ----------------------------
    dataset_train, meta_tr = load_linear_parquet_dataset(data_dir=data_dir, split='train', batch_size=batch_size, shuffle=True)
    dataset_val, meta_va = load_linear_parquet_dataset(data_dir=data_dir, split='val', batch_size=batch_size, shuffle=False)

    x_dim = int(meta_tr["x_dim"])
    y_dim = int(meta_tr["y_dim"])
    print("📦 Dataset Parquet carregado para treinamento.")

    histories = {}
    models = {}

    early_stopping = TFEarlyStopping(monitor="val_loss", patience=20, restore_best_weights=True, verbose=0)
    reduce_lr = TFReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=6, min_lr=1e-6, verbose=0)

    # ----------------------------
    # Treinamento de cada modelo
    # ----------------------------
    for name, params in configs.items():
        print(f"\n🚀 Treinando modelo {name}...")
        if params.get("linear", False):
            # Modelo Linear
            model = build_linear_model(x_dim=x_dim, y_dim=y_dim, params=params, linear=True)
        else:
            # Modelo MLP
            model = build_linear_model(x_dim=x_dim, y_dim=y_dim, params=params, linear=False)
        hist = model.fit(
            dataset_train,
            validation_data=dataset_val,
            epochs=100,
            callbacks=[early_stopping, reduce_lr],
            verbose=2
        )

        histories[name] = hist
        models[name] = model
        print(f"✅ {name} concluído - Val Loss: {min(hist.history['val_loss']):.6f}")

    # ----------------------------
    # Salvando modelos
    # ----------------------------
    for name, model in models.items():
        # Salvando modelo no path /modelos/{nome do problema}/{nome do modelo}
        save_model(model, path = f"./modelos/{problem_name}/{name}")
    
    
    return models


## Pipelines dos modelos LSTM
Implementação e uso dos preprocessors e treinadores LSTM para séries temporais (janelas seq_len e lead).

In [28]:
import os
import matplotlib.pyplot as plt
from typing import List, Tuple, Dict, Any
from tensorflow import keras
from tensorflow.keras.callbacks import EarlyStopping as TFEarlyStopping, ReduceLROnPlateau as TFReduceLROnPlateau


def lstm_preproccess_pipeline(
    data_dir: str,
    country_list: List[str],
    feature_cols: List[str],
    target_cols: List[str],
    seq_len: int,
    lead: int,
    value_cols: List[str]
) -> Tuple[LSTMPreprocessor, Dict[str, keras.Model]]:
    """
    Pipeline completa de pré-processamento (Parquet only) para LSTM,
    salvando janelas (seq_len -> lead) em Parquet.
    """

    # ----------------------------
    # Pré-processamento
    # ----------------------------
    preproc = LSTMPreprocessor(
        data_dir=data_dir,
        model_name="lstm_model",
        feature_cols=feature_cols,
        target_cols=target_cols,
        country_list=country_list,
        lag=seq_len,
        lead=lead,
    )

    preproc.load_data()
    preproc.encode(encode_cols="datetime", encode_method="time_cycle")
    preproc.encode(encode_cols="country", encode_method="label")
    preproc.split_train_val_test(train_size=0.6, val_size=0.2, test_size=0.2, time_col="datetime")
    preproc.normalize_splits(value_cols=value_cols, normalization_method="minmax")

    # Salva janelas por split direto em Parquet (método do preprocessor)
    preproc.save_splits_parquet(basename="lstm_dataset")
    print("✅ Pré-processamento sequencial concluído (Parquet).")
    return preproc


def lstm_train_pipeline(
    problem_name: str,
    data_dir: str,
    seq_len: int,
    batch_size: int = 128,
    configs: Dict[str, Dict[str, Any]] = None
) -> Tuple[LSTMPreprocessor, Dict[str, keras.Model]]:
    # ----------------------------
    # Parquet datasets
    # ----------------------------
    dataset_train, meta_tr = load_lstm_parquet_dataset(data_dir=data_dir, split="train", batch_size=batch_size, shuffle=True)
    dataset_val, meta_va = load_lstm_parquet_dataset(data_dir=data_dir, split="val", batch_size=batch_size, shuffle=False)

    x_dim = int(meta_tr["x_dim"]) ; y_dim = int(meta_tr["y_dim"]) ; seq_len_m = int(meta_tr["seq_len"]) ; lead_m = int(meta_tr["lead"]) 
    if seq_len and seq_len != seq_len_m:
        print(f"[WARN] seq_len fornecido ({seq_len}) difere do meta ({seq_len_m}). Usando meta.")
    print("📦 Dataset Parquet carregado para treinamento.")

    histories = {}
    models = {}

    early_stopping = TFEarlyStopping(monitor="val_loss", patience=20, restore_best_weights=True, verbose=0)
    reduce_lr = TFReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=6, min_lr=1e-6, verbose=0)

    for name, params in configs.items():
        print(f"\n🚀 Treinando modelo {name}...")
        model = build_lstm_model(seq_len=seq_len_m, x_dim=x_dim, y_dim=y_dim, lead=lead_m, params=params)
        hist = model.fit(
            dataset_train,
            validation_data=dataset_val,
            epochs=100,
            callbacks=[early_stopping, reduce_lr],
            verbose=2,
        )

        histories[name] = hist
        models[name] = model

        print(f"✅ {name} concluído - Val Loss: {min(hist.history['val_loss']):.6f}")

    # ----------------------------
    # Salvando modelos
    # ----------------------------
    for name, model in models.items():
        # Salvando modelo no path /modelos/{nome do problema}/{nome do modelo}
        save_model(model, path = f"./modelos/{problem_name}/{name}")
    
    
    return models


## Pipelines dos modelos TFT
Pré-processamento em parquet e treino com PyTorch Forecasting (Temporal Fusion Transformer) via Lightning.

In [29]:
import os
import pandas as pd
import torch
import torch.nn as nn
from typing import Dict, Any, List
from pytorch_forecasting import TimeSeriesDataSet
from pytorch_forecasting.models import TemporalFusionTransformer
from lightning.pytorch import Trainer, seed_everything
from lightning.pytorch.callbacks import EarlyStopping as LGEarlyStopping, LearningRateMonitor as LGLearningRateMonitor, ModelCheckpoint as LGModelCheckpoint


def tft_preproccess_pipeline(
    data_dir: str,
    country_list: List[str],
    feature_cols: List[str],
    target_cols: List[str],
    seq_len: int,
    lead: int,
    value_cols: List[str]
) -> Tuple[TFTPreprocessor, Dict[str, Any]]:
    """
    Pipeline completa de pré-processamento para TFT.
    """

    # ----------------------------
    # Pré-processamento
    # ----------------------------
    preproc = TFTPreprocessor(
        data_dir=data_dir,
        model_name='linear_model',
        feature_cols=feature_cols,
        target_cols=target_cols,
        seq_len=seq_len,
        lead=lead,
        country_list=country_list
    )

    preproc.load_data()
    preproc.encode(encode_cols='datetime', encode_method='time_cycle')
    preproc.encode(encode_cols='country', encode_method='label')
    preproc.split_train_val_test(train_size=0.8, val_size=0.1, test_size=0.1, time_col='datetime')
    preproc.normalize_splits(value_cols=value_cols, normalization_method='minmax')
    preproc.build_tft_parquets(
        group_cols=['country'],
        time_col='datetime'
    )
    print("✅ Pré-processamento tft concluído.")

    return preproc



def tft_train_pipeline(
    problem_name: str,
    data_dir: str,
    feature_cols: List[str],
    target_cols: List[str],
    seq_len: int,
    lead: int,
    batch_size: int = 128,
    configs: Dict[str, Dict[str, Any]] = None,
):
    """
    Treinamento de modelos TFT (Temporal Fusion Transformer) usando PyTorch Forecasting + Lightning.

    - Consome os parquets gerados por TFTPreprocessor: tft_dataset_train.parquet e tft_dataset_val.parquet
    - Cria TimeSeriesDataSet para treino/validação
    - Constrói o modelo via TemporalFusionTransformer.from_dataset
    - Treina com EarlyStopping e salva checkpoints por preset
    """

    # ----------------------------
    # 1. Carregar dados pré-processados via TFTPreprocessor reutilizando load_tft_dataset
    # ----------------------------
    preproc = TFTPreprocessor(
        data_dir=data_dir,
        model_name='tft_model',
        feature_cols=feature_cols,
        target_cols=target_cols,
        seq_len=seq_len,
        lead=lead,
        country_list=[],
    )

    # usa a função para retornar DataFrames — permite aplicar dtypes e criar TimeSeriesDataSet de forma consistente
    df_train = preproc.load_tft_dataset('train', target_col=target_cols[0])
    df_val = preproc.load_tft_dataset('val', target_col=target_cols[0])

    # ----------------------------
    # 2. TimeSeriesDataSet (encoder/decoder feitos internamente)
    # ----------------------------

    train_loader = df_train.to_dataloader(train=True, batch_size=batch_size, num_workers=0)
    val_loader   = df_val.to_dataloader(train=False, batch_size=batch_size, num_workers=0)

    print(f"📦 Dados TFT — batches: train={len(train_loader)} | val={len(val_loader)}")

    # ----------------------------
    # 3. Treinamento por preset
    # ----------------------------
    models = {}
    seed_everything(42)

    accelerator = "gpu" if torch.cuda.is_available() else "cpu"

    for name, params in (configs or {}).items():
        print(f"\n🚀 Treinando TFT preset: {name} [{accelerator}]")

        model = build_tft_model(
            params={
                **params,
                "dataset": df_train,
            }
        )

        save_dir = os.path.join("modelos", problem_name, "TFT", name)
        os.makedirs(save_dir, exist_ok=True)

        callbacks = [
            LGEarlyStopping(monitor="val_loss", patience=int(params.get("patience", 5)), mode="min"),
            LGLearningRateMonitor(logging_interval="epoch"),
            LGModelCheckpoint(
                dirpath=save_dir,
                filename="best",
                monitor="val_loss",
                save_top_k=1,
                mode="min",
            ),
        ]

        trainer = Trainer(
            max_epochs=int(params.get("epochs", 50)),
            accelerator=accelerator,
            devices=1,
            callbacks=callbacks,
            default_root_dir=save_dir,
            log_every_n_steps=10,
            logger=True,
            precision=32,
        )

        trainer.fit(model, train_loader, val_loader)
        print(f"✅ {name} concluído — melhor checkpoint salvo em {save_dir}")

        models[name] = model

    return models

# Capítulo 4: Definição da estrutura dos modelos - Não foi feita otimização de hiperparm
Configuração dos problemas (dados, features, janelas) e presets de hiperparâmetros para Linear/MLP, LSTM e TFT.

In [30]:
import os, gc, tensorflow as tf


gpu_devices = tf.config.list_physical_devices('GPU')
# Configurações dos modelos

configs_linear = {
    # "linear_Simple": {
    #     "linear": True,
    #     "units": [],
    #     "dropout": 0.0,
    #     "lr": 1e-3,
    #     "l2": 0.0,
    #     "layer_norm": False,
    # },
    "linear_Medium": {
        "linear": True,
        "units": [128, 64],
        "dropout": 0.1,
        "lr": 1e-3,
        "l2": 1e-6,
        "act": "relu",
        "layer_norm": False,
    },
    # "linear_Deep": {
    #     "linear": False,
    #     "units": [256, 128, 64],
    #     "dropout": 0.2,
    #     "lr": 1e-3,
    #     "l2": 1e-6,
    #     "act": "relu",
    #     "layer_norm": True,
    # },
}

configs_mlp = {
    # "mlp_Simple": {
    #     "linear": False,
    #     "units": [64],
    #     "dropout": 0.05,
    #     "lr": 1e-3,
    #     "l2": 1e-6,
    #     "act": "relu",
    #     "layer_norm": False,
    # },
    "mlp_Medium": {
        "linear": False,
        "units": [128, 64],
        "dropout": 0.1,
        "lr": 1e-3,
        "l2": 1e-6,
        "act": "relu",
        "layer_norm": False,
    },
    # "mlp_Deep": {
    #     "linear": False,
    #     "units": [256, 128, 64],
    #     "dropout": 0.2,
    #     "lr": 1e-3,
    #     "l2": 1e-6,
    #     "act": "relu",
    #     "layer_norm": True,
    # },
}

configs_lstm = {
    # "lstm_Simple": {
    #     "lstm_units": [64],
    #     "dense_units": [64],
    #     "dropout": 0.05,
    #     "rec_dropout": 0.0,
    #     "act": "tanh",
    #     "lr": 1e-3,
    #     "l2": 1e-6,
    #     "layer_norm": False,
    # },
    "lstm_Medium": {
        "lstm_units": [128, 64],
        "dense_units": [64],
        "dropout": 0 if gpu_devices else 0.15,
        "rec_dropout": 0 if gpu_devices else 0.05,
        "act": "tanh",
        "lr": 1e-3,
        "l2": 1e-6,
        "layer_norm": True,
    },
}

# Presets de TFT compatíveis com build_tft_model (PyTorch Forecasting)
# Campos utilizados: hidden_size, dropout, lstm_layers, num_heads, lr, epochs, patience
config_tft = {
    # "tft_Simple": {
    #     "hidden_size": 64,
    #     "dropout": 0.1,
    #     "lstm_layers": 1,
    #     "num_heads": 4,
    #     "lr": 1e-3,
    #     "epochs": 50,
    #     "patience": 5,
    # },
    "tft_Medium": {
        "hidden_size": 64,
        "lstm_layers": 1,
        "num_heads": 2,
        "dropout": 0.2,
        "hidden_continuous_size": 32,
        "attention_head_size": 2,
        "learning_rate": 1e-3,
        "patience": 20,
        "epochs": 50,
    }, 
    
    # "tft_Deep": {
    #     "hidden_size": 256,
    #     "dropout": 0.2,
    #     "lstm_layers": 2,
    #     "num_heads": 8,
    #     "lr": 1e-3,
    #     "epochs": 100,
    #     "patience": 10,
    # },
}


# Capítulo 5: Préprocessamento de dados

In [31]:
import concurrent.futures
import tensorflow as tf
import gc


preprocess_collector = {}
def run_preprocessing(cfg):
    """Executa o pipeline completo de pré-processamento para um problema."""
    name = cfg["name"]
    try:
        print(f"\n🚀 Executando Preprocessamento do problema {name} ...")

        print("🧠 Pré-processando dados do modelo linear/MLP")
        preproc_lin = linear_preproccess_pipeline(
            data_dir=cfg["data_dir"],
            feature_cols=cfg["feats"],
            target_cols=cfg["tgts"],
            lag=cfg["lag"],
            lead=cfg["lead"],
            value_cols=cfg["vals"],
            country_list=cfg["countries"],
            reduced_dim=cfg.get("reduced_dim", None),
            mask_value=cfg.get("mask_value", -999.0),
        )

        del preproc_lin
        tf.keras.backend.clear_session()
        gc.collect()

        print("🧠 Pré-processando dados do modelo LSTM")
        preproc_lstm = lstm_preproccess_pipeline(
            data_dir=cfg["data_dir"],
            feature_cols=cfg["feats"],
            target_cols=cfg["tgts"],
            seq_len=cfg["lag"],
            lead=cfg["lead"],
            value_cols=cfg["vals"],
            country_list=cfg["countries"],
        )
        del preproc_lstm
        tf.keras.backend.clear_session()
        gc.collect()

        # === Adicionado: pré-processamento TFT ===
        print("🧠 Pré-processando dados do modelo TFT")
        preproc_tft = tft_preproccess_pipeline(
            data_dir=cfg["data_dir"],
            feature_cols=cfg["feats"],
            target_cols=cfg["tgts"],
            seq_len=cfg["lag"],
            lead=cfg["lead"],
            value_cols=cfg["vals"],
            country_list=cfg["countries"],
        )
        del preproc_tft
        tf.keras.backend.clear_session()
        gc.collect()

        print(f"✅ Finalizado {name} - memória liberada\n{'-'*60}")
        return (name, "OK")

    except Exception as e:
        print(f"❌ Erro em {name}: {e}")
        return (name, f"ERRO: {e}")


# ================================
# Execução paralela
# ================================
MAX_WORKERS = min(8, len(problemas))  # ajuste conforme núcleos / VRAM disponível

with concurrent.futures.ThreadPoolExecutor(max_workers=MAX_WORKERS) as executor:
    futures = [executor.submit(run_preprocessing, cfg) for cfg in problemas]

    for future in concurrent.futures.as_completed(futures):
        name, status = future.result()
        print(f"🧩 Resultado {name}: {status}")



🚀 Executando Preprocessamento do problema treinamento ...
🧠 Pré-processando dados do modelo linear/MLP

🚀 Executando Preprocessamento do problema N1A ...
🧠 Pré-processando dados do modelo linear/MLP

🚀 Executando Preprocessamento do problema N1B ...
🧠 Pré-processando dados do modelo linear/MLP

🚀 Executando Preprocessamento do problema N1C ...
🧠 Pré-processando dados do modelo linear/MLP

🚀 Executando Preprocessamento do problema N2A ...
🧠 Pré-processando dados do modelo linear/MLP

🚀 Executando Preprocessamento do problema N2B ...
🧠 Pré-processando dados do modelo linear/MLP

🚀 Executando Preprocessamento do problema N2C ...
🧠 Pré-processando dados do modelo linear/MLP
[DIVIDIDO] train: 27,931 linhas
[DIVIDIDO] val: 3,491 linhas
[DIVIDIDO] test: 3,492 linhas
[DIVIDIDO] train: 27,931 linhas
[DIVIDIDO] val: 3,491 linhas
[DIVIDIDO] test: 3,492 linhas
[DIVIDIDO] train: 27,931 linhas
[DIVIDIDO] val: 3,491 linhas
[DIVIDIDO] test: 3,492 linhas
[ℹ️] Reduzindo lags: usando 168 e mascarando 72

# Capítulo 6: Treinamento dos modelos
Este capítulo executa, por problema: Linear/MLP (configs_linear), MLP (configs_mlp), LSTM (configs_lstm) e TFT (config_tft), liberando memória entre execuções.

In [ ]:
import tensorflow as tf
import gc
# Loop de treinamento sequencial: Linear/MLP -> MLP -> LSTM -> TFT

tempo_treino = {}

# Carrega configuração de treinamento
cfg = []
for tmp in problemas:
    if tmp["name"] == "treinamento":
        cfg = tmp
        break
    else:
        continue

if not cfg:
    print("sem configuração de 'treinamento' configurada na lista de problemas")
else:
    name = cfg["name"]
    print(f"\n🚀 Iniciando treinamento dos modelos ...")

    print("="*80)
    print("Modelo Linear...")
    # Treinamento Linear
    try:
        tempo_treino["linear"] = {}
        tempo_treino["linear"]["inicio"] = time.time()
        models_linear = linear_train_pipeline(
            problem_name=name,
            data_dir=cfg["data_dir"],
            batch_size=256,
            configs=configs_linear,
        )
        del models_linear
        tempo_treino["linear"]["fim"] = time.time()
        tempo_treino["linear"]["duracao"] = tempo_treino["linear"]["fim"] - tempo_treino["linear"]["inicio"]
    except Exception as e:
        print(f"❌ Erro ao treinar Linear para {name}: {e}")
    finally:
        tf.keras.backend.clear_session()
        gc.collect()

    print("="*80)
    print("Modelo MLP...")
    # Treinamento MLP (configs_mlp)
    try:
        tempo_treino["mlp"] = {}
        tempo_treino["mlp"]["inicio"] = time.time()
        models_mlp = linear_train_pipeline(
            problem_name=name,
            data_dir=cfg["data_dir"],
            batch_size=256,
            configs=configs_mlp,
        )
        del models_mlp
        tempo_treino["mlp"]["fim"] = time.time()
        tempo_treino["mlp"]["duracao"] = tempo_treino["mlp"]["fim"] - tempo_treino["mlp"]["inicio"]
    except Exception as e:
        print(f"❌ Erro ao treinar MLP (configs_mlp) para {name}: {e}")
    finally:
        tf.keras.backend.clear_session()
        gc.collect()

    print("="*80)
    print("Modelo LSTM...")
    # Treinamento LSTM
    try:
        tempo_treino["lstm"] = {}
        tempo_treino["lstm"]["inicio"] = time.time()
        models_lstm = lstm_train_pipeline(
            problem_name=name,
            data_dir=cfg["data_dir"],
            seq_len=cfg.get("lag") or cfg.get("seq_len"),
            batch_size=256,
            configs=configs_lstm,
        )
        del models_lstm
        tempo_treino["lstm"]["fim"] = time.time()
        tempo_treino["lstm"]["duracao"] = tempo_treino["lstm"]["fim"] - tempo_treino["lstm"]["inicio"]
    except Exception as e:
        print(f"❌ Erro ao treinar LSTM para {name}: {e}")
    finally:
        tf.keras.backend.clear_session()
        gc.collect()


    print("="*80)
    print("Modelo TFT...")
    # Treinamento TFT (Temporal Fusion Transformer)
    try:
        tempo_treino["tft"] = {}
        tempo_treino["tft"]["inicio"] = time.time()
        models_tft = tft_train_pipeline(
            problem_name=name,
            data_dir=cfg["data_dir"],
            feature_cols=cfg.get("feats") or cfg.get("feature_cols"),
            target_cols=cfg.get("tgts") or cfg.get("target_cols"),
            seq_len=cfg.get("lag"),
            lead=cfg.get("lead"),
            batch_size=256,
            configs=config_tft,
        )
        # libera referência ao retorno (modelos serializados internamente)
        del models_tft
        tempo_treino["tft"]["fim"] = time.time()
        tempo_treino["tft"]["duracao"] = tempo_treino["tft"]["fim"] - tempo_treino["tft"]["inicio"]
    except Exception as e:
        print(f"❌ Erro ao treinar TFT para {name}: {e}")

    finally:
        tf.keras.backend.clear_session()
        gc.collect()

    print(f"✅ Problema {name} concluído — memória limpa\n{'-'*60}")
    for modelo, tempo in tempo_treino.items():
        duracao = tempo.get("duracao", 0)
        print(f"⏱️  Tempo de treino {modelo}: {duracao:.2f} segundos")


🚀 Iniciando treinamento dos modelos ...
Modelo LSTM...
📦 Dataset Parquet carregado para treinamento.

🚀 Treinando modelo lstm_Medium...
Epoch 1/100
245/245 - 9s - 38ms/step - loss: 0.0620 - mae: 0.2041 - val_loss: 0.0479 - val_mae: 0.1831 - learning_rate: 0.0010
Epoch 2/100
245/245 - 6s - 24ms/step - loss: 0.0570 - mae: 0.1982 - val_loss: 0.0475 - val_mae: 0.1822 - learning_rate: 0.0010
Epoch 3/100
245/245 - 6s - 24ms/step - loss: 0.0570 - mae: 0.1983 - val_loss: 0.0480 - val_mae: 0.1834 - learning_rate: 0.0010
Epoch 4/100
245/245 - 6s - 24ms/step - loss: 0.0569 - mae: 0.1982 - val_loss: 0.0476 - val_mae: 0.1825 - learning_rate: 0.0010
Epoch 5/100
245/245 - 6s - 24ms/step - loss: 0.0569 - mae: 0.1982 - val_loss: 0.0477 - val_mae: 0.1827 - learning_rate: 0.0010
Epoch 6/100
245/245 - 6s - 24ms/step - loss: 0.0569 - mae: 0.1982 - val_loss: 0.0480 - val_mae: 0.1835 - learning_rate: 0.0010
Epoch 7/100
245/245 - 6s - 24ms/step - loss: 0.0569 - mae: 0.1982 - val_loss: 0.0471 - val_mae: 0.181